**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# Your code here to find the table name
tables = list(client.list_tables(dataset))
for item in tables:
    print(item.table_id)

taxi_trips


In [4]:
# Write the table name as a string below
table_name = "taxi_trips"

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
#q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

In [6]:
# Your code here
# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,5ca0f04de0e5c08a254cd55ae0ffd107491f21c9,93ed0b8d2841e33df894a893a95c6287c2b58779b0529d...,2015-05-10 10:30:00+00:00,2015-05-10 11:00:00+00:00,1949,9.0,None,None,14,None,...,0.0,23.05,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
1,07ec06d200f3d9199e450d6fb527519f12bbebc8,0fb1abb17fe80c09b7eb931aaa40b4157514f6466a12e2...,2015-05-03 04:15:00+00:00,2015-05-03 04:30:00+00:00,1553,17.3,None,None,14,None,...,1.0,41.92,Credit Card,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
2,c430712943435faed8227777e0836ed9f4419038,46cb4badd64238b521d799f0aeb48c0214ebcac417ea25...,2015-04-29 21:45:00+00:00,2015-04-29 22:15:00+00:00,1541,5.3,None,None,14,None,...,1.0,15.85,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
3,fd026dfd0a58c63707625cf3b3d5cc5f32b33720,c0233d1acdf95f778faf4bcadcff7035dc6bbcab99f07e...,2015-05-02 09:45:00+00:00,2015-05-02 10:00:00+00:00,1509,9.1,None,None,14,None,...,0.0,27.31,Credit Card,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
4,97b677642b2bdc18e47b5803e81053c6533cf0b0,f705c825446b6687d3b45d5bb04a7a29720306923943f3...,2015-05-09 07:00:00+00:00,2015-05-09 07:15:00+00:00,584,3.1,None,None,14,None,...,0.0,9.45,Cash,Yellow Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None


After deciding whether you see any important issues, run the code cell below.

### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT EXTRACT(YEAR FROM trip_start_timestamp)</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [7]:
# Your code goes here
rides_per_year_query =  """
                        SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        GROUP BY year
                        ORDER BY COUNT(1) DESC
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
#rides_per_year_query_job = ____ # Your code goes here
c = client.query(rides_per_year_query, job_config=safe_config)
# View top few rows of results

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = c.to_dataframe() # Your code goes here
# View results
print(rides_per_year_result)
# Check your answer
q_3.check()

NameError: name 'cc' is not defined

### 4) Dive slightly deeper

You'd like to take a closer look at rides from 2017.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2017.
2. Modify the query to extract the month rather than the year.

In [ ]:
# Your code goes here
rides_per_year_query =  """
                        SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month,  COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
                        GROUP BY month
                        ORDER BY COUNT(1) DESC
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
#rides_per_year_query_job = ____ # Your code goes here
c = client.query(rides_per_year_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = c.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [ ]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
c = client.query(speeds_query, job_config=safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = c.to_dataframe() # Your code here

# View results
print(speeds_result)

# Check your answer
q_5.check()

For the solution, uncomment the appropriate line below.

That's a hard query. If you made good progress towards the solution, congratulations!

# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the Intro to SQL micro-course.

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*